### MediaPipe

In [3]:
import cv2
import glob
import os
import mediapipe as mp

def detect_face(path, save_path, split_num):
    count_num = 0  # 카운트 숫자 초기화

    split_num = split_num

    # 저장할 폴더 경로
    folder_path = save_path

    # 저장 폴더가 없다면 생성
    if not os.path.exists(f'{folder_path}'):
        os.makedirs(folder_path, exist_ok=True)

    # 얼굴 검출기
    mp_face_detection = mp.solutions.face_detection

    last_folder_name = None  # 이전 폴더명 초기값

    for face_path in glob.glob(path, recursive=True):
        path_split = face_path.split('\\')
        print(path_split[-1])

        # 저장할 폴더 이름 초기화
        folder_name = path_split[split_num]

        if path_split[-1] == '':
            path_split.remove('')

        try:
            if path_split[-1].split('.')[-1].lower() == 'jpg':
                # 이미지 로드
                image = cv2.imread(face_path)

                # 얼굴 검출 실행
                with mp_face_detection.FaceDetection(model_selection=1, min_detection_confidence=0.5) as face_detection:
                    results = face_detection.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

                # 얼굴 검출된 영역 추출하여 저장
                for detection in results.detections:
                    
                    # 얼굴 부분만 잘라내기
                    relative_bounding_box = detection.location_data.relative_bounding_box
                    ymin = int(relative_bounding_box.ymin * image.shape[0])
                    xmin = int(relative_bounding_box.xmin * image.shape[1])
                    ymax = int((relative_bounding_box.ymin + relative_bounding_box.height) * image.shape[0])
                    xmax = int((relative_bounding_box.xmin + relative_bounding_box.width) * image.shape[1])
                    face_crop = image[ymin - 200:ymax + 200, xmin - 200:xmax + 200]

                    if folder_name != last_folder_name:
                        count_num = 0  # 카운트 숫자 초기화
                        if not os.path.exists(f'{folder_path}/{folder_name}'): # 폴더가 없다면                         
                            os.makedirs(f'{folder_path}/{folder_name}', exist_ok=True)  # 폴더 생성
                            print(f'{folder_name} 폴더 생성')

                        # 얼굴 이미지를 JPG 파일로 저장
                        count_num += 1
                        cv2.imwrite(f'{folder_path}/{folder_name}/{count_num}.jpg', face_crop)
                        print('저장 완료')

                    else:
                        # 얼굴 이미지를 JPG 파일로 저장
                        count_num += 1
                        cv2.imwrite(f'{folder_path}/{folder_name}/{count_num}.jpg', face_crop)
                        print('동일 인물 추가 사진 저장 완료')
                
                # 마지막 폴더명 저장
                last_folder_name = folder_name

            else:
                print('이미지 파일이 아님')
        except Exception as e:
            print(e)
            pass


In [4]:
path = "D:\workspace_kitae\MainProject\ImageProject\VS1 (2)\\VS1\\**"
save_path = "D:\workspace_kitae\MainProject\ImageProject\VS1 (2)\\VS1\\test"
detect_face(path, save_path, 6) 
# detect_face(변환할 폴더 경로, 저장할 폴더 경로 (없으면 생성됨), 사람마다 구분된 폴더명 인덱스번호)


이미지 파일이 아님
test
이미지 파일이 아님


### Haar

In [ ]:
import cv2
import glob
import os

def detect_face(path, save_path):
    # 저장할 폴더 경로
    folder_path = save_path

    # 추가 인식된 얼굴 저장 경로 폴더 만들기
    os.makedirs(f'{folder_path}/over_idx', exist_ok=True)

    # 얼굴 인식을 위한 Haar Cascade XML 파일 경로
    cascade_path = './xml/haarcascade_frontalface_default.xml'

    # Haar Cascade 분류기 초기화
    face_cascade = cv2.CascadeClassifier(cascade_path)

    for face_path in glob.glob(path, recursive=True):

        count_num = 0

        path_split = face_path.split('\\')

        # 저장할 폴더 이름
        folder_name = path_split[7] 
        
        if path_split[-1] == '':
            path_split.remove('')

        try:         
            if path_split[-1].split('.')[-1].lower() == 'jpg':

                # 이미지 로드
                image = cv2.imread(face_path)
                
                # 그레이스케일로 변환
                gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            
                # 얼굴 검출 실행
                faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=10, minSize=(20, 20))
              
                # 얼굴 검출된 영역 추출하여 저장
                for idx, (x, y, w, h) in enumerate(faces):
                    face_image = image[y:y+h, x:x+w]

                    if idx == 1:
                        # 폴더가 없음
                        if not os.path.exists(folder_path):
                            count_num = 0 # 카운트 숫자 초기화
                            os.makedirs(f'C:/Users/tjoeun/Desktop/faces/{folder_name}', exist_ok=True) # 폴더 생성
                            print(f'{folder_name}폴더 생성')

                            # 얼굴 이미지를 JPG 파일로 저장
                            cv2.imwrite(f'C:/Users/tjoeun/Desktop/faces/{folder_name}/{count_num+1}.jpg', face_image)
                            print('폴더 생성 후 저장 완료')
                        
                        # 폴더가 존재
                        else:                       
                            # 얼굴 이미지를 JPG 파일로 저장
                            count_num += 1
                            cv2.imwrite(f'C:/Users/tjoeun/Desktop/faces/{folder_name}/{count_num}.jpg', face_image)
                            print('이미 폴더 생성됨. 저장 완료')
                            
                    else:
                        # 폴더가 없음
                        if not os.path.exists(folder_path):
                            os.makedirs(f'C:/Users/tjoeun/Desktop/faces/over_idx/{folder_name}', exist_ok=True) # 폴더 생성
                            print(f'추가 안면 사진 {folder_name}폴더 생성')

                            # 얼굴 이미지를 JPG 파일로 저장
                            cv2.imwrite(f'C:/Users/tjoeun/Desktop/faces/over_idx/{folder_name}/{idx}.jpg', face_image)
                            print('폴더 생성 후 추가 안면 사진 저장 완료')

                        
                        # 폴더가 존재
                        else:                            
                            # 얼굴 이미지를 JPG 파일로 저장
                            cv2.imwrite(f'C:/Users/tjoeun/Desktop/faces/over_idx/{folder_name}/{idx}.jpg', face_image)
                            print('추가 안면 사진 폴더. 이미 생성됨. 저장 완료')

            else:
                print('not image file')
        except Exception as e:
            print(e)
            pass

In [ ]:
path = "D:\\Face\\facedata\\data\\2.Validation\\real\\**\\**"
detect_face(path)